##Mentiras, malditas mentiras, y encuestas (*)


####Guillermo Moncecchi (@gmonce)

Época de elecciones, época de encuestas. Y época de análisis de encuestas. Hace tiempo (mucho tiempo) que tengo la misma impresión: me parece que la mayoría de las afirmaciones que hacen los politólogos (y, mucho más, los medios) son, directamente erróneas, o al menos no se desprenden de los datos. Van dos encuestas en las que baja un punto y entonces: "El Frente Amplio consolida su caída". Pero, claro, mi afirmación es tan imprecisa como las originales. 

Aunque trabajo habitualmente con probabilidades, no soy ni de lejos un experto (siquiera un conocedor profundo) de los métodos estadísticos. Así que leí un poco, busqué otro poco... y finalmente encontré exactamente lo que quería: este artículo sobre cómo analizar resultados de encuestas, y cómo tener en cuenta el "margen de error" reportado. Si les interesa leerlo, se llama "The 'Margin of Error' for Differences in Polls", de Charles Franklin, y está disponible en https://abcnews.go.com/images/PollingUnit/MOEFranklin.pdf. 

Algunos comentarios, antes de empezar: 

- Una encuesta es una consulta a un grupo de gente, esperando que sea representativa del total de la población (en este caso, los votantes). 
- El número que se publica es la proporción de votantes que eligió a cada partido (por ejemplo: 0.32 para el Partido Nacional quiere decir que 32 de cada 100 encuestados tomó esa opción). 
- Aademás, se publica un "margen de error" (del tipo +/- x%) y un "nivel de confianza" (típicamente del 95%), que dice lo siguiente (leer atentamente, porque es larguito): si hacemos 100 encuestas como éstas, en 95 de ellas, el número va a estar entre el valor de la proporción más/menos el valor del margen de error. A esto se le llama usualmente _intervalo de confianza_. Por ejemplo, si el margen de error es +/- 3,2%, y el nivel de confianza es el 95%,  el 0.32 se transforma en un rango entre 0.288 y 0.352 (expresado usualmente como [0.288,0.352]). 
- Si leemos nuevamente lo anterior, vemos que quiere decir, nada más ni nada menos, que una de cada 20 veces que haga esta encuesta me voy a equivocar y el número va a salirse del rango. _No quiere decir_ que haya un 95% de certeza (y por eso el término "confianza" no es muy adecuado) sobre los resultados.
- Cuando un valor cae dentro del intervalo de confianza, decimos que es _estadísticamente significativo_, lo cual quiere decir que no es probable que haya sido por azar.

O sea, a las encuestas hay que tomarlas con pinzas. No voy a repetirlo, pero cada vez que demos un valor o un intervalo, recuerden que 1 de cada 20 veces es normal errarle. Normal. 

Vamos a los datos para ver ejemplos: consideramos la encuesta Factum en el 2009 en diversas ediciones (elijo la misma encuesta para que las comparaciones sean válidas, porque uno supone que el método es el mismo):

In [0]:
# Datos
# Consideramos los votos a setiembre de diferentes años, para ver cómo van cambiando
votaciones_factum_2014={'votoFA':0.42,'votoPN':0.32,'votoPC':0.15,'votoPI':0.03,'votoIndefinidos':0.04,'votoOtros':0.02}
votaciones_factum_julio_2014={'votoFA':0.42,'votoPN':0.30,'votoPC':0.14,'votoPI':0.03,'votoIndefinidos':0.04,'votoOtros':0.02}
votaciones_factum_2013={'votoFA':0.43,'votoPN':0.23,'votoPC':0.16,'votoPI':0.02,'votoIndefinidos':0.08,'votoOtros':0.08}
votaciones_factum_2010={'votoFA':0.49,'votoPN':0.22,'votoPC':0.13,'votoPI':0.00,'votoIndefinidos':0.00,'votoOtros':0.00}
votaciones_factum_2009={'votoFA':0.46,'votoPN':0.34,'votoPC':0.10,'votoPI':0.02,'votoIndefinidos':0.06,'votoOtros':0.02}


La biblioteca franklin.py (en Python) tiene la definición de las funciones que vamos a utilizar. Para los que les interese el código para jugar con sus propios números, están disponibles [aquí](https://github.com/gmonce/datascience/blob/master/src/franklin.py), pero quieren ser exactamente las fórmulas mencionadas en el paper. *Desafío para programadores con más habilidades (y tiempo) que yo: hacer una página web que haga estas cuentas para cualquier par de valores en las encuestas*

In [0]:
from franklin import *

Empecemos por ver los diferentes intervalos de confianza para cada partido, según la última encuesta (Setiembre de 2014):

In [32]:
# Veamos los intervalos de confianza para los votos al día de hoy
for (key,value) in votaciones_factum_2014.items():
    print (key,votaciones_factum_2014[key],ci(votaciones_factum_2014[key]))

votoFA 0.42 (0.389, 0.451)
votoPN 0.32 (0.291, 0.349)
votoPC 0.15 (0.128, 0.172)
votoPI 0.03 (0.019, 0.041)
votoIndefinidos 0.04 (0.028, 0.052)
votoOtros 0.02 (0.011, 0.029)


Por ejemplo, el Frente Amplio está entre 0.389 y 0.451. El Partido Colorado está entre 0.128 y 0.17. Pero hay que tener en cuenta que estos rangos aplican a la proporción calculada, es decir al valor obtenido por cada partido en particular. No puede usarse ese rango para comparar valores (por cuestiones de varianza, errores estándar y esas cosas de estadísticos). Para eso, hay que hacer algunas cuentas (que están descritas en el paper, y programadas más arriba). Veamos algunos ejemplos: 

*Pregunta 1: ¿Es estadísticamente significativa la diferencia entre el FA y el PN?* Esta pregunta puede traducirse como "¿podemos afirmar con confianza que el FA tiene más votos que el PN? (siempre dentro del 95% mencionado, que dije que no iba a mencionar de nuevo, pero no puedo evitarlo...). Calculemos el intervalo para la diferencia entre ambos votos. Si el rango no incluye al 0, entonces la diferencia es significativa:

In [33]:
cidif=ci_dif(votaciones_factum_2014['votoFA'],votaciones_factum_2014['votoPN'])

0.42 0.32 Diferencia: 0.09999999999999998
Intervalo: (0.046, 0.154)
La diferencia es significativa


Es. Estamos bastante seguros de que (según las encuestas), va ganando el FA. _Pregunta 2: ¿Es significativa la diferencia entre el FA y los partidos tradicionales sumados?_

In [34]:
cidif=ci_dif(votaciones_factum_2014['votoFA'],votaciones_factum_2014['votoPN']+votaciones_factum_2014['votoPC'])

0.42 0.47 Diferencia: -0.04999999999999999
Intervalo: (-0.109, 0.009)
La diferencia no es significativa


Primera observación: si blancos y colorados suman sus votos, _no sabemos qué pasa_. Atención cuando digo que no sabemos qué pasa, no estoy hablando de indecisos. Estoy diciendo que con la cantidad de gente que encuestamos, los modelos estadísticos en las que nos basamos nos dicen que no alcanza para hacer la afirmación, al menos para no errarle en más de 1 en 20 veces.

También podemos comparar resultados entre diferentes encuestas, para ver si hubo cambios. Con  _otra_ fórmula. _Pregunta 3:¿Mejoró el Partido Nacional del 2010 hasta ahora?_  

In [35]:
cidif=ci_dif_between(votaciones_factum_2010['votoPN'],votaciones_factum_2014['votoPN'])

0.22 0.32 Diferencia -0.1
Intervalo: (0.061, 0.139)
La diferencia es significativa


Sí, mejoró. ¿Y el Partido Colorado?

In [36]:
cidif=ci_dif_between(votaciones_factum_2010['votoPC'],votaciones_factum_2014['votoPC'])


0.13 0.15 Diferencia -0.01999999999999999
Intervalo: (-0.011, 0.051)
La diferencia no es significativa


No, ¿y el FA?

In [37]:
cidif=ci_dif_between(votaciones_factum_2010['votoFA'],votaciones_factum_2014['votoFA'])

0.49 0.42 Diferencia 0.07
Intervalo: (-0.114, -0.026)
La diferencia es significativa


El FA está hoy peor que luego de ganar las elecciones. Pero tal vez sería mejor comparar con el 2009 (es decir, _antes_ de las elecciones, en la misma época en la que estamos ahora). De hecho, podemos comparar a todos los partidos. 

In [38]:
for partido in ['votoFA','votoPN','votoPC','votoPI']:
    print (partido)
    cidif=ci_dif_between(votaciones_factum_2009[partido],votaciones_factum_2014[partido])


votoFA
0.46 0.42 Diferencia 0.040000000000000036
Intervalo: (-0.084, 0.004)
La diferencia no es significativa
votoPN
0.34 0.32 Diferencia 0.020000000000000018
Intervalo: (-0.062, 0.022)
La diferencia no es significativa
votoPC
0.1 0.15 Diferencia -0.04999999999999999
Intervalo: (0.021, 0.079)
La diferencia es significativa
votoPI
0.02 0.03 Diferencia -0.009999999999999998
Intervalo: (-0.004, 0.024)
La diferencia no es significativa


Lo único que podríamos afirmar con "certeza estadística" es que el Partido Colorado está mejor que en las elecciones pasadas. Después, todo más o menos igual

En los diarios uruguayos se hacen afirmaciones sobre las encuestas que parecen omitir el margen de error. Si un partido bajó un punto, "su imagen se ha deteriorado". Si sube dos, "muestra un repunte". La mayoría de esas afirmaciones son estadísticamente _muy_ arriesgadas. Veamos un ejemplo: 

_Blancos y Colorados sumados siguen aventajando al Frente Amplio_ El País 8/9/2014. http://www.elpais.com.uy/informacion/encuesta-factum-intencion-voto-septiembre.html

In [39]:
cidif=ci_dif(votaciones_factum_2014['votoFA'],votaciones_factum_2014['votoPN']+votaciones_factum_2014['votoPC'])

0.42 0.47 Diferencia: -0.04999999999999999
Intervalo: (-0.109, 0.009)
La diferencia no es significativa


Como dijimos antes, los números son demasiado parecidos como para afirmar que este valor no es casualidad. En la misma nota: _"el Frente Amplio detuvo la caída"_

In [40]:
cidif=ci_dif_between(votaciones_factum_julio_2014['votoFA'],votaciones_factum_2014['votoFA'])

0.42 0.42 Diferencia 0.0
Intervalo: (-0.044, 0.044)
La diferencia no es significativa


No sabemos. De hecho no sabemos si efectivamente venía cayendo, porque la diferencia entre los números es siempre muy pequeña. En la misma nota: _"El partido Nacional sigue creciendo"_

In [41]:
cidif=ci_dif_between(votaciones_factum_julio_2014['votoPN'],votaciones_factum_2014['votoPN'])

0.3 0.32 Diferencia -0.020000000000000018
Intervalo: (-0.021, 0.061)
La diferencia no es significativa


Tampoco.

Estimado lector: la próxima vez que veas afirmaciones (especialmente sobre diferencias), desconfía. Lo más probable es que, salvo que la diferencia sea muy grande, el margen de error producido por encuestar a 900 personas sea tan grande que haga que las afirmaciones no sean estadísticamente adecuadas. Muchas veces, terminan explicando el ruido estadístico. Y eso es malo, porque disfraza de ciencia lo que son especulaciones (es válido especular; lo que no es valido es disfrazarlo de ciencia). 



_El código presentado en este notebook es tomado del artículo mencionado, si detectan algún error por favor me avisan a @gmonce en Twitter._

(*) Esto no es un juicio de valor, solamente un juego de palabras. http://en.wikipedia.org/wiki/Lies,_damned_lies,_and_statistics